In [7]:
cd C:\Users\stfu_\OneDrive\Desktop\BigData\lab1\my-mle-template

C:\Users\stfu_\OneDrive\Desktop\BigData\lab1\my-mle-template


In [49]:
import configparser
config = configparser.ConfigParser()

config['data.raw'] = {
    'data1': 'data/winequality-red.csv',
    'data2': 'data/winequality-white.csv',
    'sep': ';',
    'train_pers': '0.8',
}
config['data.prep'] = {
    'train_x': 'data/train_X.csv',
    'train_y': 'data/train_y.csv',
    'test_x': 'data/test_X.csv',
    'test_y': 'data/test_y.csv',
}

config['random_forest'] = {
    'n_estimators': 100,
    'criterion': 'gini',
    'random_state': 42,
    'save_path': 'experiments/rf.joblib'
}

with open('config.ini', 'w') as configfile:
  config.write(configfile)

In [ ]:
import configparser
import pandas as pd
from sklearn.model_selection import train_test_split


if __name__ == '__main__':
    config = configparser.ConfigParser()
    config.sections()
    config.read('config.ini')

    data = []
    sep = config['data.raw']['sep']
    for k, path in config['data.raw'].items():
        if 'data' in k:
            data.append(pd.read_csv(path, sep=sep))
    data = pd.concat(data)
    X_train, X_test, y_train, y_test = train_test_split(
        data.iloc[:, :-1], data.iloc[:, -1],
        train_size=float(config['data.raw']['train_pers']), random_state=42
    )
    X_train.to_csv(config['data.prep']['train_x'], index=False)
    y_train.to_csv(config['data.prep']['train_y'], index=False)
    X_test.to_csv(config['data.prep']['test_x'], index=False)
    y_test.to_csv(config['data.prep']['test_y'], index=False)

In [77]:
import configparser
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from joblib import dump
import pickle

if __name__ == '__main__':
    config = configparser.ConfigParser()
    config.sections()
    config.read('config.ini')

    X_train = pd.read_csv(config['data.prep']['train_x'])
    y_train = pd.read_csv(config['data.prep']['train_y'])
    X_test = pd.read_csv(config['data.prep']['test_x'])
    y_test = pd.read_csv(config['data.prep']['test_y'])

    clf = RandomForestClassifier(n_estimators=int(config['random_forest']['n_estimators']),
                                 random_state=int(config['random_forest']['random_state']),)
    print('Training...')
    clf = clf.fit(X_train, y_train.iloc[:,0])
    print('Saving model...')
    dump(clf, config['random_forest']['save_path'])

    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy:\t{accuracy}')

Training...
Saving model...
Accuracy:	0.67
